# Introduction

A short notebook where we explore hair removal techniques on dermatoscopic images

## Import libraries

In [14]:
# Basic libraries
import os
import warnings
import cv2
from tqdm import tqdm 
import random
import numpy as np

import matplotlib.pyplot as plt

## Load data

In [2]:
# from google.colab import drive
# drive.mount("/content/gdrive")

In [2]:
IMAGE_FOLDER = "/Users/mipopovic/Desktop/MunjeLumenDS2025/data/test_output/train/benign/"
MASK_FOLDER = "/Users/mipopovic/Desktop/MunjeLumenDS2025/data/segmentation_mask/"
# RESIZED_FOLDER = "/Users/mipopovic/Desktop/MunjeLumenDS2025/data/sample_images/resized/"

In [4]:
os.makedirs(IMAGE_FOLDER, exist_ok=True)
os.makedirs(MASK_FOLDER, exist_ok=True)

# Util functions

In [5]:
def show_images_in_line(folder_path, num_images=5, image_width=100, image_height=100):
    """
    Displays images from a folder in a single row using Matplotlib.

    :param folder_path: Path to the folder containing images.
    :param num_images: Number of images to display.
    :param image_width: Width to resize each image.
    :param image_height: Height to resize each image.
    """
    # Get list of image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    
    # Limit to the desired number of images
    image_files = image_files[:num_images]

    if not image_files:
        print("No images found in the folder.")
        return

    fig, axes = plt.subplots(1, len(image_files), figsize=(len(image_files) * 3, 3))

    # If only one image, make axes a list for uniformity
    if len(image_files) == 1:
        axes = [axes]

    for ax, file in zip(axes, image_files):
        img_path = os.path.join(folder_path, file)
        img = cv2.imread(img_path)  # Read image with OpenCV
        if img is None:
            continue  # Skip unreadable images
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for Matplotlib
        img = cv2.resize(img, (image_width, image_height))  # Resize for consistency
        ax.imshow(img)
        ax.set_title(file)
        ax.axis("off")

    plt.tight_layout()
    plt.show()

In [10]:
def process_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists
    file_list = os.listdir(input_folder)  # Get all files in the folder

    for file_name in tqdm(file_list, desc="Processing Images", unit="file"):
        image_path = os.path.join(input_folder, file_name)
        image = cv2.imread(image_path)

        if image is None:
            continue

        # Process the resized image
        enhanced_image = apply_clahe(image)
        hair_mask = detect_hair(enhanced_image)
        clean_image = remove_hair(image, hair_mask)

        output_path = os.path.join(output_folder, file_name)
        cv2.imwrite(output_path, clean_image)  # Save processed image

    print("✅ Batch Processing Complete!")



# Defining functions

In [36]:
image_path = IMAGE_FOLDER + "a.jpg"
mask_path = MASK_FOLDER + "a.png"

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

In [37]:
# Ensure mask has the same size as image
if mask.shape[:2] != image.shape[:2]:
    mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

# Re-threshold after resizing to ensure binary values (0 or 1)
_, mask = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)

In [38]:
skin_pixels = image[mask == 0]  # shape: (N_skin_pixels, 3)


In [39]:
# Convert whole image to LAB
image_lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

# Extract LAB values of skin pixels
skin_lab = image_lab[mask == 0]

# Compute mean LAB values
mean_L, mean_A, mean_B = np.mean(skin_lab, axis=0)


In [40]:
# ITA = arctangent((L - 50) / B) in degrees
L = skin_lab[:, 0].astype(np.float32)
B = skin_lab[:, 2].astype(np.float32)
ita = np.arctan2((L - 50), B) * 180 / np.pi

mean_ita = np.mean(ita)

In [41]:
def classify_skin_tone(ita_val):
    if ita_val > 55:
        return "Very Light"
    elif ita_val > 41:
        return "Light"
    elif ita_val > 28:
        return "Intermediate"
    elif ita_val > 10:
        return "Tan"
    elif ita_val > -30:
        return "Brown"
    else:
        return "Dark"



In [42]:
skin_tone_category = classify_skin_tone(mean_ita)


In [ ]:
def which_color_skin(image, mask):
    # Ensure mask has the same size as image
    if mask.shape[:2] != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    # Re-threshold after resizing to ensure binary values (0 or 1)
    _, mask = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)

    skin_pixels = image[mask == 0]  # shape: (N_skin_pixels, 3)

    # Convert whole image to LAB
    image_lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    
    # Extract LAB values of skin pixels
    skin_lab = image_lab[mask == 0]
    
    # Compute mean LAB values
    mean_L, mean_A, mean_B = np.mean(skin_lab, axis=0)

    # ITA = arctangent((L - 50) / B) in degrees
    L = skin_lab[:, 0].astype(np.float32)
    B = skin_lab[:, 2].astype(np.float32)
    ita = np.arctan2((L - 50), B) * 180 / np.pi
    
    mean_ita = np.mean(ita)

    skin_tone_category = classify_skin_tone(mean_ita)
    
    return skin_tone_category



## Processing and Evaluation

In [43]:
print(skin_tone_category)

Light


## Conclusion